<a href="https://colab.research.google.com/github/MRdvan/Pytorch-Training/blob/master/Pytorch-fundamentals-to-deploy/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [5]:
# YOUR CODE HERE
%%writefile get_data.py
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it... 
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...") 
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")


Writing get_data.py


In [7]:
# Example running of get_data.py
!python get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [20]:
# YOUR CODE HERE
import os
import torch
import data_setup,engine,model_builder,utils

from torchvision import transforms

#hyperparameters
num_epochs = 5
batch_size = 25
hidden_units = 10
learning_rate = 0.001

train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

device = "cuda" if torch.cuda.is_available() else "cpu"

#transform
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])
#loader
train_loader,test_loader,classes = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = data_transform,
    batch_size = batch_size
)
#model
model = model_builder.TinyVGG(
    input_shape = 3,hidden_units = hidden_units,output_shape = len(classes)
).to(device)

#loss & optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(),lr = learning_rate)

#train
results = engine.train(
    model=model,train_dataloader = train_loader,test_dataloader = test_loader,optimizer = optimizer,loss_fn = loss_fn,epochs = num_epochs,device= device
)

#save results
utils.save_model(model,"models","05_going_modular_script_mode_tinyvgg_model.pth")


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1013 | train_acc: 0.2667 | test_loss: 1.0994 | test_acc: 0.2533
Epoch: 2 | train_loss: 1.0957 | train_acc: 0.3333 | test_loss: 1.0973 | test_acc: 0.2533
Epoch: 3 | train_loss: 1.0885 | train_acc: 0.3644 | test_loss: 1.0998 | test_acc: 0.2533
Epoch: 4 | train_loss: 1.0616 | train_acc: 0.4622 | test_loss: 1.0727 | test_acc: 0.3467
Epoch: 5 | train_loss: 1.0095 | train_acc: 0.5644 | test_loss: 1.0387 | test_acc: 0.3600
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


In [15]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.0885 | train_acc: 0.4414 | test_loss: 1.1006 | test_acc: 0.2708
 20% 1/5 [00:02<00:11,  2.85s/it]Epoch: 2 | train_loss: 1.0477 | train_acc: 0.4375 | test_loss: 1.1292 | test_acc: 0.2708
 40% 2/5 [00:04<00:05,  1.97s/it]Epoch: 3 | train_loss: 1.0214 | train_acc: 0.4883 | test_loss: 1.1099 | test_acc: 0.3011
 60% 3/5 [00:05<00:03,  1.69s/it]Epoch: 4 | train_loss: 1.0037 | train_acc: 0.5430 | test_loss: 1.1279 | test_acc: 0.3229
 80% 4/5 [00:06<00:01,  1.56s/it]Epoch: 5 | train_loss: 0.9635 | train_acc: 0.5234 | test_loss: 1.0390 | test_acc: 0.3116
100% 5/5 [00:08<00:00,  1.66s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [18]:
# YOUR CODE HERE
%%writefile predict.py

import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import model_builder
from typing import List, Tuple
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_PATH = "data/pizza_steak_sushi/test/sushi/175783.jpg"

img = Image.open(IMAGE_PATH)
model = model_builder.TinyVGG(input_shape = 3,hidden_units = 10,output_shape = 3).to(device)


image_transform = transforms.Compose(
    [
        transforms.Resize(size=(64,64)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
    ]
)



model.eval()
with torch.inference_mode():
  transformed_img = image_transform(img).unsqueeze(dim=0)
  transformed_img.to(device)
  logit = model(transformed_img)

class_pred = torch.softmax(logit,dim=1).argmax(dim=1)
plt.figure()
plt.imshow(img)
plt.title(
  f"pred: {class_pred}"
)
plt.axis(False)


Overwriting predict.py


In [19]:
# Example running of predict.py 
!python predict.py --image_path "data/pizza_steak_sushi/test/sushi/175783.jpg"